Question 2

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from dask import delayed
from dask import compute
import dask.array as da
from statistics import mean
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
#import statsmodels.api as sm
from scipy.stats import gaussian_kde
import warnings
warnings.filterwarnings("ignore")

In [19]:
#Loading the data
df = pd.read_csv('data/Auto.csv', na_values='?')
df.dropna(inplace=True)
df.columns=['mpg', 'cyl', 'displ', 'hpwr', 'wgt', 'accl', 'yr', 'orgn','name']
df.head()

#Creating the dummy variables
df['mpg_high'] = 0
df.mpg_high[df.mpg>=df.mpg.median()] = 1
df['orgn1'] = (df.orgn==1).astype(int)
df['orgn2'] = (df.orgn==2).astype(int)
df.head()

,mpg,cyl,displ,hpwr,wgt,accl,yr,orgn,name,mpg_high,orgn1,orgn2
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu,0,1,0
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320,0,1,0
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite,0,1,0
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst,0,1,0
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino,0,1,0


In [20]:
%%time
N_bs = 100

err_vector = np.zeros(N_bs)
X = df[['cyl', 'displ', 'hpwr', 'wgt', 'accl', 'yr', 'orgn1', 'orgn2']].values
y = df.mpg_high.values

for bs_indic in range(N_bs):
    X_train, X_test, y_train, y_test = \
            train_test_split(X, y, test_size=0.35, random_state=100+bs_indic)
    LogReg = LogisticRegression(n_jobs = None)
    LogReg.fit(X_train, y_train)
    y_pred = LogReg.predict(X_test)
    err_vector[bs_indic] = (y_test == y_pred).mean()
    
mean_err = err_vector.mean()

print('The average error rate =', mean_err)

The average error rate = 0.8976086956521739
Wall time: 243 ms


In [21]:
num_cores = multiprocessing.cpu_count()
print('Number of available cores is', num_cores)

Number of available cores is 4


In [22]:
def paral_bs(bs_indic, seed, X, y):    
    X_train, X_test, y_train, y_test = \
            train_test_split(X, y, test_size=0.35, random_state=seed+bs_indic)
    LogReg = LogisticRegression(n_jobs = None)
    LogReg.fit(X_train, y_train)
    y_pred = LogReg.predict(X_test)
    return (y_test == y_pred).mean()

In [23]:
%%time
N_bs = 100
err_vector = []
X = df[['cyl', 'displ', 'hpwr', 'wgt', 'accl', 'yr', 'orgn1', 'orgn2']].values
y = df.mpg_high.values

for bs_indic in range(N_bs):
    sample = delayed(paral_bs(bs_indic, 100, X, y))
    err_vector.append(sample)

mean_err = delayed(mean)(err_vector)
mean_err = compute(mean_err)

print('The average error rate =', mean_err[0])

The average error rate = 0.8976086956521739
Wall time: 343 ms
